# Imports

In [1]:
#export
from bayes_opt import BayesianOptimization, UtilityFunction, SequentialDomainReductionTransformer
import numpy as np
from pprint import pprint

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

# Code

In [55]:
#export
def mean_duplicates(points):
    uni_params = {}
    targets = {}
    for i, p in enumerate(points):
        param, target = p['params'], p['target']
        for k,v in uni_params.items():
            if param == v:
                targets[k].append(target)
                break
        else:
            uni_params[i] = param
            targets[i] = [target]

    res = []
    for k, v in uni_params.items():
        res.append({'params':v, 'target':np.mean(targets[k])})

    return res


class BaseConfigBo:
    def __init__(self):
        self.utility = UtilityFunction(kind="ei", kappa=1, xi=0.1)
        
    def get_values(self, optimizer):
        return optimizer.suggest(self.utility)
    
    def init_opt(self, bounds):
        return BayesianOptimization(f=None,
                                    pbounds=bounds,
                                   random_state=1)
                                    #bounds_transformer=SequentialDomainReductionTransformer())
        
    def register(self, opt, points):
        for point in points:
            opt.register(
                params=point['params'],
                target=point['target'],)
        
    def init_map(self):
        raise NotImplementedError
    
    def read_params(self, params):
        bounds = {}
        for p in params:
            bounds[p['name']] = p['bounds']
        return bounds
    
    def create_state(self, points, params):
        bounds = self.read_params(params)
        o = self.init_opt(bounds)
        if points:
            points = mean_duplicates(points)
            #pprint(points, indent=4)
            self.register(o, points)
        print(o.res)
        print([self.get_values(o) for i in range(10)])
        new_params = self.get_values(o)
        params_map = self.init_map()
        cfg = {}
        for name, (full_name, p_type, default_value) in params_map.items():
            value = new_params.get(name, default_value)
            if value is not np.NaN:
                cfg[full_name] = p_type(value)
        return new_params, cfg

# Tests

In [56]:
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

In [57]:
class Bo(BaseConfigBo):
    def init_map(self):
        return {
            'g':('generations', int, 2),
            'e':('exp_power', int, np.NaN),
            'f0':('dec_f0', int, np.NaN),
            'f1':('dec_f1', int, np.NaN),
            'f2':('dec_f2', int, np.NaN),
            'f3':('dec_f3', int, np.NaN),
            'mc':('mutate_chance', float, np.NaN),
            'cr':('crossover_chance', float, np.NaN),
            'co':('combine_chance', float, np.NaN)
        }

bo = Bo()
p1 = {'name':'e', 'bounds':(1,15)}
p2 = {'name':'cr', 'bounds':(.01,.99)}

def bo_all(**kwargs):
    inner_state, new_state=bo.create_state(kwargs['params'], [p1, p2])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def bo_exp(**kwargs):
    inner_state, new_state=bo.create_state(kwargs['params'], [p1])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)


In [58]:
points = [{'params':{'e':1},'target':0.1},
          {'params':{'e':1},'target':0.2}
         ]
bo.create_state(points, [p1])

({'e': 10.822801232517557}, {'generations': 2, 'exp_power': 10})

In [64]:
# points = [{'params':{'e':1,  'a':2},'target':0.1},
#           {'params':{'e':1,  'a':2},'target':0.2},
#           {'params':{'e':1,'a':2},'target':0.5},
#           {'params':{'e':1.2,'d':2},'target':0.0},
#           {'params':{'e':330,'k':2},'target':0.2}
#          ]

points = [
    {'params': {'cr': 0.9885257859302491}, 'target': 0.577763631939888},
    {'params': {'cr': 0.9885660456872435}, 'target': 0.5584937930107117},
    {'params': {'cr': 0.010119437482166772}, 'target': 0.5518008545041084},
    {'params': {'cr': 0.0101559066291012}, 'target': 0.5431795716285706},
    {'params': {'cr': 0.9899404424902588}, 'target': 0.5732786804437637},
    {'params': {'cr': 0.9893949599413431}, 'target': 0.5737223923206329},
    {'params': {'cr': 0.9898670833454175}, 'target': 0.5675030797719955},
    {'params': {'cr': 0.3092491872609939}, 'target': 0.5605851411819458},
    {'params': {'cr': 0.6836879323470609}, 'target': 0.567135363817215},
    {'params': {'cr': 0.5175112773164423}, 'target': 0.5583183616399765},
    {'params': {'cr': 0.010109605687386128}, 'target': 0.5333066955208778}
]

In [65]:
from random import shuffle

In [66]:
p1 = {'name':'cr', 'bounds':(0.01,.99)}
shuffle(points)

In [71]:
bo.create_state(points, [p1])

({'cr': 0.987307333997757},
 {'generations': 2, 'crossover_chance': 0.987307333997757})

In [63]:
ts = [bo.create_state(points, [p1])[0]['cr'] for i in range(100)]
np.mean(ts)

0.9874239338671746

In [87]:
np.arange(10)/10

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [14]:
points = [#[0.0101, 0.075 ],
       [0.0102, 0.061 ],
       [0.0101, 0.062 ],
       [0.9894, 0.574 ],
       #[0.9899, 0.568 ],
       [0.9899, 0.573 ],
       [0.6837, 0.567 ],
       [0.3092, 0.561 ],
       [0.5175, 0.558 ]]
rs = []
for p in reversed(points):
    rs.append(({'x':p[0]}, p[1]))

In [467]:
def gen_x():
    x = list(range(1,10))
    t = list(range(1,10))
    x = x+[.1]
    t = t+[0]
    x = x+[9.9] 
    t = t+[-.2]
    rs = []
    for i,j in zip(x,t):
        yield {'x':i}, j/10

In [468]:
bounds = {'x':(0.1, 9.9)}
utility = UtilityFunction(kind="ei", kappa=None, xi=0.1)
optimizer = BayesianOptimization(
    f=None,
    pbounds=bounds,
    verbose=2,
    random_state=1)
g = gen_x()

In [535]:
x, t = next(g)
optimizer.register(
    params=x,
    target=t)
print(x,t)

{'x': 7} 0.7


In [517]:
utility = UtilityFunction(kind="ei", kappa=100, xi=0)

In [572]:
optimizer.suggest(utility)

{'x': 5.100098810089408}

In [563]:
optimizer.register(
    params=6.68,
    target=.6)

In [578]:
optimizer._space.target

array([ 0.1, -0.2,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7, -2. ,  0.4,  0.6])

In [579]:
optimizer.res

[{'target': 0.1, 'params': {'x': 1.0}},
 {'target': -0.2, 'params': {'x': 0.901}},
 {'target': 0.2, 'params': {'x': 2.0}},
 {'target': 0.3, 'params': {'x': 3.0}},
 {'target': 0.4, 'params': {'x': 4.0}},
 {'target': 0.5, 'params': {'x': 5.0}},
 {'target': 0.6, 'params': {'x': 6.0}},
 {'target': 0.7, 'params': {'x': 7.0}},
 {'target': -2.0, 'params': {'x': 7.6}},
 {'target': 0.4, 'params': {'x': 5.1}},
 {'target': 0.6, 'params': {'x': 6.68}}]

In [15]:

for x, t in rs:
    optimizer.register(
    params=x,
    target=t)
    print(optimizer.suggest(utility), x, t)
    #break

NameError: name 'utility' is not defined

In [ ]:
utility = UtilityFunction(kind='ei',kappa=None, xi=.1)

sg = [optimizer.suggest(utility)['x'] for i in range(100)]

In [83]:
optimizer.res

[{'target': 0.558, 'params': {'x': 0.5175}}]

In [ ]:
np.array(sg).std()

In [ ]:
plt.hist(sg, bins=50);

In [ ]:
plt.plot(optimizer.space.target, label='Mutated Optimizer')

# More Tests

In [582]:
from skopt import Optimizer
from skopt.space import Real
from joblib import Parallel, delayed
# example objective taken from skopt
from skopt.benchmarks import branin

In [ ]:
optimizer = Optimizer()

In [660]:
optimizer = Optimizer(
    dimensions=[Real(1,15), Real(0.01,0.99)],
    random_state=1,
    base_estimator='gp'
)

In [661]:
for i in range(10):
    x = optimizer.ask(n_points=4)
    y = Parallel(n_jobs=4)(delayed(branin)(v) for v in x)
    optimizer.tell(x, y)

In [662]:
min(optimizer.yi)

1.8727364911380562

In [655]:
min(optimizer.yi)

1.8727364911380562

In [645]:
optimizer.get_result()

          fun: 9.887668349173717
    func_vals: array([23.33345935, 20.49882368,  9.88766835, 18.34898225])
       models: []
 random_state: RandomState(MT19937) at 0x7F79A313B888
        space: Space([Real(low=1, high=15, prior='uniform', transform='normalize'),
       Real(low=0.01, high=0.99, prior='uniform', transform='normalize')])
        specs: None
            x: [7.904456632264042, 0.8811468968722667]
      x_iters: [[10.837029894221025, 0.07041869717793878], [6.276486266433995, 0.15273052748956706], [7.904456632264042, 0.8811468968722667], [1.8188206027183147, 0.26058092147083095]]

In [ ]:
for i in range(10):
    x = optimizer.ask(n_points=4)  # x is a list of n_points points
    y = Parallel(n_jobs=4)(delayed(branin)(v) for v in x)  # evaluate points in parallel
    optimizer.tell(x, y)

# takes ~ 20 sec to get here
print(min(optimizer.yi))  # print the best objective found

# Exports

In [1]:
!python3 extra/n2s.py bo.ipynb

Converted bo.ipynb to exp/nb_bo.py
